In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [2]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [3]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

/Users/dylansteinhauer/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/Users/dylansteinhauer/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1  = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

2022-03-14 10:50:35.530296: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
35/35 [==============================] - 2s 6ms/step - loss: 403.7263 - accuracy: 0.2595
Epoch 2/100
35/35 [==============================] - 0s 4ms/step - loss: 82.9902 - accuracy: 0.6343
Epoch 3/100
35/35 [==============================] - 0s 4ms/step - loss: 47.3038 - accuracy: 0.6525
Epoch 4/100
35/35 [==============================] - 0s 4ms/step - loss: 31.8748 - accuracy: 0.6661
Epoch 5/100
35/35 [==============================] - 0s 4ms/step - loss: 22.5499 - accuracy: 0.6906
Epoch 6/100
35/35 [==============================] - 0s 4ms/step - loss: 16.1862 - accuracy: 0.6987
Epoch 7/100
35/35 [==============================] - 0s 4ms/step - loss: 11.4480 - accuracy: 0.7187
Epoch 8/100
35/35 [==============================] - 0s 5ms/step - loss: 8.3720 - accuracy: 0.7405
Epoch 9/100
35/35 [==============================] - 0s 3ms/step - loss: 6.8573 - accuracy: 0.7550
Epoch 10/100
35/35 [==============================] - 0s 3ms/step - loss: 5.5008 - accuracy: 0.7387
E

35/35 [==============================] - 0s 7ms/step - loss: 0.6702 - accuracy: 0.8140
Epoch 83/100
35/35 [==============================] - 0s 4ms/step - loss: 0.6744 - accuracy: 0.8067
Epoch 84/100
35/35 [==============================] - 0s 6ms/step - loss: 0.8377 - accuracy: 0.8049
Epoch 85/100
35/35 [==============================] - 0s 6ms/step - loss: 0.8147 - accuracy: 0.8049
Epoch 86/100
35/35 [==============================] - 0s 4ms/step - loss: 1.2462 - accuracy: 0.8113
Epoch 87/100
35/35 [==============================] - 0s 4ms/step - loss: 1.6824 - accuracy: 0.7731
Epoch 88/100
35/35 [==============================] - 0s 4ms/step - loss: 1.6647 - accuracy: 0.7604
Epoch 89/100
35/35 [==============================] - 0s 4ms/step - loss: 1.2389 - accuracy: 0.7750
Epoch 90/100
35/35 [==============================] - 0s 4ms/step - loss: 1.2359 - accuracy: 0.7877
Epoch 91/100
35/35 [==============================] - 0s 4ms/step - loss: 0.8427 - accuracy: 0.7922
Epoch 92/100


In [11]:
# Evaulate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 1s - loss: 0.5702 - accuracy: 0.8043 - 500ms/epoch - 42ms/step
Loss: 0.5702328681945801, Accuracy: 0.804347813129425


In [13]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
33/35 [===========================>..] - ETA: 0s - loss: 4.7507 - accuracy: 0.7557
Epoch 1: saving model to checkpoints/weights.01.hdf5
35/35 [==============================] - 1s 6ms/step - loss: 4.6603 - accuracy: 0.7495
Epoch 2/100
34/35 [============================>.] - ETA: 0s - loss: 1.1348 - accuracy: 0.7831
Epoch 2: saving model to checkpoints/weights.02.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 1.1326 - accuracy: 0.7804
Epoch 3/100
31/35 [=========================>....] - ETA: 0s - loss: 0.8084 - accuracy: 0.7974
Epoch 3: saving model to checkpoints/weights.03.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 0.7892 - accuracy: 0.7949
Epoch 4/100
34/35 [============================>.] - ETA: 0s - loss: 1.5966 - accuracy: 0.7941
Epoch 4: saving model to checkpoints/weights.04.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 1.6010 - accuracy: 0.7895
Epoch 5/100
17/35 [=============>................] - ETA: 0s

Epoch 36/100
24/35 [===================>..........] - ETA: 0s - loss: 0.9625 - accuracy: 0.8477
Epoch 36: saving model to checkpoints/weights.36.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 1.1411 - accuracy: 0.8131
Epoch 37/100
23/35 [==================>...........] - ETA: 0s - loss: 1.1166 - accuracy: 0.8207
Epoch 37: saving model to checkpoints/weights.37.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 1.0996 - accuracy: 0.7967
Epoch 38/100
21/35 [=================>............] - ETA: 0s - loss: 1.4860 - accuracy: 0.7812
Epoch 38: saving model to checkpoints/weights.38.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 1.8786 - accuracy: 0.7768
Epoch 39/100
23/35 [==================>...........] - ETA: 0s - loss: 1.6579 - accuracy: 0.7622
Epoch 39: saving model to checkpoints/weights.39.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 1.3700 - accuracy: 0.7858
Epoch 40/100
21/35 [=================>............] 

35/35 [==============================] - 0s 3ms/step - loss: 0.7376 - accuracy: 0.8131
Epoch 71/100
19/35 [===============>..............] - ETA: 0s - loss: 0.5746 - accuracy: 0.8388
Epoch 71: saving model to checkpoints/weights.71.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 1.0501 - accuracy: 0.8113
Epoch 72/100
21/35 [=================>............] - ETA: 0s - loss: 1.6053 - accuracy: 0.7515
Epoch 72: saving model to checkpoints/weights.72.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 1.3676 - accuracy: 0.7895
Epoch 73/100
23/35 [==================>...........] - ETA: 0s - loss: 0.7274 - accuracy: 0.8274
Epoch 73: saving model to checkpoints/weights.73.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.6552 - accuracy: 0.8258
Epoch 74/100
35/35 [==============================] - ETA: 0s - loss: 0.6820 - accuracy: 0.8194
Epoch 74: saving model to checkpoints/weights.74.hdf5
35/35 [==============================] - 0s 4ms/step

In [20]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.4328 - accuracy: 0.8016 - 277ms/epoch - 23ms/step
Loss: 0.43275949358940125, Accuracy: 0.801630437374115


In [21]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [22]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [23]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.4328 - accuracy: 0.8016 - 170ms/epoch - 14ms/step
Loss: 0.43275949358940125, Accuracy: 0.801630437374115
